In [2]:
import zipfile
import pandas as pd
import matplotlib as plt
#pd.set_option('display.max_columns', 70)
import warnings
warnings.filterwarnings('ignore')


In [3]:
zf = zipfile.ZipFile("input/archive_nps.zip")
df = pd.read_parquet(zf.open('nps_data_full.parquet'),engine='fastparquet')

zf.close()

In [4]:
df_1 = df[['FB_UID','WEEK_COMMENCING','STORE_LOCATIONCLASS','RATING_NPS','FB_SENTIMENT','FB_CHANNEL']]

In [5]:
df_1['WEEK_COMMENCING'] = pd.to_datetime(df_1['WEEK_COMMENCING'],yearfirst=True)

In [6]:
# Group by day and calculate NPS for each day
grouped_data = df_1.groupby(df_1['WEEK_COMMENCING'].dt.date)

# Initialize empty lists to store NPS scores for each day
nps_scores = []

# Loop through each day's data
for day, group in grouped_data:
    sample_size = group['FB_UID'].count()
    promoter = group[group['RATING_NPS'] >= 9]['FB_UID'].count()
    detractor = group[group['RATING_NPS'] <= 6]['FB_UID'].count()

    # Calculate NPS for the current day
    percentage_promoters = (promoter / sample_size) * 100
    percentage_detractors = (detractor / sample_size) * 100

    NPS = percentage_promoters - percentage_detractors

    # Store the NPS score and the corresponding date in a list
    nps_scores.append({'Date': day, 'NPS': NPS})

# Create a new DataFrame with NPS scores for each day
nps_df = pd.DataFrame(nps_scores)

In [7]:
nps_df

,Date,NPS
0,2019-12-30,42.234332
1,2020-01-06,36.246068
2,2020-01-13,32.385195
3,2020-01-20,22.180569
4,2020-01-27,19.018405
...,...,...
136,2022-08-08,50.962627
137,2022-08-15,46.710890
138,2022-08-22,52.720348
139,2022-08-29,49.664776


In [8]:
df_1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 451591 entries, 0 to 451590
Data columns (total 6 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   FB_UID               451591 non-null  object        
 1   WEEK_COMMENCING      451591 non-null  datetime64[ns]
 2   STORE_LOCATIONCLASS  451591 non-null  object        
 3   RATING_NPS           450958 non-null  float64       
 4   FB_SENTIMENT         451591 non-null  object        
 5   FB_CHANNEL           451588 non-null  object        
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 20.7+ MB


In [33]:
pd.DataFrame(df_1['FB_SENTIMENT'].value_counts()).reset_index()

,FB_SENTIMENT,count
0,Promoter,286234
1,Detractor,117872
2,Passive,46852
3,Unrated,633
